

### 1. Functions as return values
### 2. The operator module
### 3. Decorators
### 4. Decorators with arguments

## 1.

### Functions as arguments

#### Lets start with our factorial function 

In [1]:
l_factorial = lambda n:1 if n == 0 else n*l_factorial(n-1)

#### Timing
##### The procedural way, going line by line
Factorial is a recursive and hence time-consuming operation. Lets see how long it takes

In [2]:
import time

t0 = time.time()
l_factorial(900)
t1 = time.time()
print('Took %.8f s'%(t1-t0))

Took 0.00099730 s


#### The functional way, with a wrapper function
But a better way is to write a wrapper function that times every function that's passed onto it!

In [3]:
def timer(func, arg):

    t0 = time.time()
    func(arg)
    t1 = time.time()
    return t1-t0

print('Took %.8f s'%timer(l_factorial, 900))

Took 0.00099874 s


#### The fully functional way, with lambda wrapper functions
We can even turn timer()into a lambda function, although it takes a pretty functional state of mind to do so

In [4]:
l_timestamp = lambda func, arg:(time.time(), func(arg), time.time())
l_diff = lambda t0, retval, t1:t1-t0
l_timer = lambda func, arg:l_diff(*l_timestamp(func, arg))

print('Took %.8f s'%l_timer(l_factorial, 900))

Took 0.00099826 s


### Nested functions
- How to define nested functions
- How nested functions affect variable scope
- How to control variable scope

#### Knowest thy scope
#### Inner and outer functions
- Let's start by defining a very basic nested function
- lets all function refer to a variable x. This is the same variable, the global variable x, in all cases

In [5]:
def outer():

    x = 'outer'
    def inner():
        
        x = 'inner'
        print('Inner:\t\t', x)
    
    print('Outer (before):', x)
    inner()
    print('Outer  (after):', x)
    

x = 'global'
print('Global (before):', x)
outer()
print('Global (after):', x)

Global (before): global
Outer (before): outer
Inner:		 inner
Outer  (after): outer
Global (after): global


### Controlling the variable scope with global and nonlocal
- global binds a variable to the global level
- nonlocal (Python >=3) binds a variable to one level higher

In [6]:
def outer():

    x = 'outer'
    def inner():
        
        global x
        print('Inner:\t\t', x)
    
    print('Outer (before):', x)
    inner()
    print('Outer  (after):', x)
    

x = 'global'
print('Global (before):', x)
outer()
print('Global (after):', x)

Global (before): global
Outer (before): outer
Inner:		 global
Outer  (after): outer
Global (after): global


In [7]:
def outer():

    x = 'outer'
    def inner():
        
        nonlocal x
        print('Inner:\t\t', x)
    
    print('Outer (before):', x)
    inner()
    print('Outer  (after):', x)
    

x = 'global'
print('Global (before):', x)
outer()
print('Global (after):', x)

Global (before): global
Outer (before): outer
Inner:		 outer
Outer  (after): outer
Global (after): global


#### Four steps to baking a (pre-baked ) croissant
- We need to perform four steps

In [8]:
preheat_oven = lambda: print('Preheating oven')
put_croissants_in = lambda: print('Putting croissants in')
wait_five_minutes = lambda: print('Waiting five minutes')
take_croissants_out = lambda: print('Take croissants out(and eat them!)')

#### Passing all steps to a launcher function
Alternatively, we can create a launcher function(peform_recipe( ) to which w ass all functions and which then performs all these functions for us. This is, by itself. not very useful

In [9]:
def perform_steps(*functions):
    
    for function in functions:
        function()
    
perform_steps(preheat_oven,
    put_croissants_in,
    wait_five_minutes,
    take_croissants_out)

Preheating oven
Putting croissants in
Waiting five minutes
Take croissants out(and eat them!)


### Wrapping all steps into a single recipe

In [10]:
def perform_steps(*functions):
    
    def run_all():
        
        for function in functions:
            function()
    return run_all

    
run = perform_steps(preheat_oven,
    put_croissants_in,
    wait_five_minutes,
    take_croissants_out)

run()

Preheating oven
Putting croissants in
Waiting five minutes
Take croissants out(and eat them!)


## 2.

### The operator module

In [11]:
l_factorial = lambda n:1 if n == 0 else n*l_factorial(n-1)

#### Chaining functions and combining return values
- Say that we want to call this function a number of times, with different arguments, and do something with the return values. How can we do that?

In [12]:
def chain_mul(*what):
    
    '''Takes a List of (function, argument )tuples. Calls each
    function with its argument, multiplies up the return values
    (starting at 1) and returns the total'''
    total = 1
    for (func, arg) in what:
        total *= func(arg)
    return total

chain_mul((l_factorial, 2), (l_factorial, 3))    

12

#### Operators as regular functions
- The function above is not very general: it can only multiple values, not multiply or subtract them. Ideally, we would pass an operator to the function as well
- But * is syntax an not an object that we can pass! Fortunately the Pythons built-in **operator module** offers all operators as regular functions

In [13]:
import operator

def chain(how, *what):
    
    total = 1
    for (func, arg) in what:
        total = how(total, func(arg))
    return total

chain(operator.truediv, (l_factorial, 2), (l_factorial, 3))  

0.08333333333333333

## 3.

### That's a decorator!

In [14]:
import time

factorial = lambda n:1 if n == 0 else n*factorial(n-1)

def timer(func):

    def inner(arg):
        
        t0 = time.time()
        func(arg)
        t1 = time.time()
        return t1-t0
    
    return inner


timed_factorail = timer(factorial)
timed_factorail(900)

0.0

- The timer function that we've defined above is a decorator. You can apply a decorator to a function directly, using the @ syntax

In [15]:
@timer
def timed_factorial(n):
    
    return 1 if n == 0 else n*factorial(n-1)

print(timed_factorial(900))

0.0009965896606445312


## 4.
### Decorators with arguments

- What if we want to specify the units of time?
If we want to specify the units of time(seconds or milliseconds), we need to prove the units of time as arguments to the decorator.
- We can do this, but it requires another level of nesting

In [16]:
import time

factorial = lambda n:1 if n == 0 else n*factorial(n-1)

def timer_with_arguments(units='s'):

    def timer(func):

        def inner(arg):

            t0 = time.time()
            func(arg)
            t1 = time.time()
            diff = t1-t0
            if units == 'ms':
                diff *= 1000
            return diff

        return inner
    
    return timer

timed_factorial = timer(factorial)
print(timed_factorial(900))
timed_factorial = timer_with_arguments(units='ms')(factorial)
print(timed_factorial(1000))

0.0009984970092773438
0.9980201721191406


#### That's a decorator with arguments
- Again, using the @ syntax, this is gives very clean code!

In [17]:
@timer_with_arguments(units='s')
def a_factorail(n):
    
    return 1 if n == 0 else n*a_factorail(n-1)

a_factorail(900)

0.000997781753540039